In [32]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from pathlib import Path
import chromadb

load_dotenv()
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")
knowledge_base_paths = Path("KnowledgeBase")# 
files_path = [f for f in knowledge_base_paths.iterdir() if f.is_file()]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
files_path

[PosixPath('KnowledgeBase/Diyanet-Vakfi-Yayinlari-ilmihal.pdf')]

In [33]:
def create_vectorDB(collection_name, db_path, embedding_function,chunks):
    vectorstore = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding_function,
    collection_name=collection_name, 
    persist_directory=db_path,
    collection_metadata={"embeeding_model":"text-embedding-3-large",
    "chunk_size":1024,
    "chunk_overlap":100})
    return vectorstore

In [34]:
def is_collection_vorhanden(col_name, client):
    col_list = [ item.name for item in client.list_collections()]
    if col_name in col_list:
        return False
    else: return True

In [35]:
for path in files_path:
    client = chromadb.PersistentClient("vectorDB")
    docs = PyPDFLoader(path).load() 
    collection_name = str(path).split("/")[1]
    if is_collection_vorhanden(path, client):
        chunks = splitter.split_documents(docs)
        
        create_vectorDB(collection_name, "vectorDB", embedding_function, chunks)
        print(f"collectin {collection_name} wurde  mit {client.get_collection(collection_name).count()} erstellt")
    else:
        print("collection ist schon vorhanden")
        
    

collectin Diyanet-Vakfi-Yayinlari-ilmihal.pdf wurde  mit 3681 erstellt


In [36]:
vectorstore = Chroma( 
embedding_function=embedding_function,
persist_directory="vectorDB",
collection_metadata={"embeeding_model":"text-embedding-3-large",
"chunk_size":1024,
"chunk_overlap":100})

In [37]:
import chromadb
client = chromadb.PersistentClient("vectorDB")

In [38]:
client.list_collections()

[Collection(name=asrin-getirdigi-tereddutler-1-M-F-Gulen.pdf),
 Collection(name=RUYA-ISTIHARE-TEFEUl-sorularla-Islamiyet.pdf),
 Collection(name=MUZIK_VE_ILAHI_sorularla_islamiyet.pdf),
 Collection(name=PEYGAMBERLER-Sorularla-Islamiyet.pdf),
 Collection(name=TAKVA-sorularla-Islamiyet.pdf),
 Collection(name=MIRAS_sorularla_islamiyet.pdf),
 Collection(name=ZEKAT-sorularla-Islamiyet.pdf),
 Collection(name=Asrin-Getirdigi-Tereddutler-2-M-F-Gulen.pdf),
 Collection(name=TASAVVUF-VE-TARIKAT-sorularla-Islamiyet.pdf),
 Collection(name=YEMIN-KEFARET-sorularla-Islamiyet.pdf),
 Collection(name=ZAMAN-sorularla-Islamiyet.pdf),
 Collection(name=VATAN-DEVLET-MILLET-sorularla-Islamiyet.pdf),
 Collection(name=Asrin-Getirdigi-Tereddutler-4-M-F-Gulen.pdf),
 Collection(name=langchain),
 Collection(name=SAHABELER-sorularla-Islamiyet.pdf),
 Collection(name=TICARET-sorularla-Islamiyet.pdf),
 Collection(name=Bir-Muslumanin-Yol-Haritasi-Akademi-AHeyeti.pdf),
 Collection(name=RUHLAR-sorularla-Islamiyet.pdf),
 Col

In [39]:
embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore_vectorDB = Chroma( 
client=client,
embedding_function=embedding_function,
persist_directory="vectorDB",
collection_metadata={"embeeding_model":"text-embedding-3-large",
"chunk_size":1024,
"chunk_overlap":100})

In [40]:
test_data = ["Allah her şeyi yarattı, -hâşâ- O’nu kim yarattı?",
"Nürnbergde hava nasil?",
"Allah’ın özü ve nitelikleri nelerdir?",
"sigara içmek haram mıdır?",
"safii mezhebine göre vücuttan gelen kan abdesti bozar mi?"]

In [41]:
collection_list = client.list_collections()

ergebnisse=[]
query_vek = embedding_function.embed_query(test_data[4])
for col_name in collection_list:
    name=col_name.name
    print(name)
    col = client.get_collection(name=name)
    res = col.query(query_vek,n_results=3)
    print(res)
    ergebnisse.append(res)


asrin-getirdigi-tereddutler-1-M-F-Gulen.pdf
{'ids': [['aa2cbd09-d738-4ba1-8cf9-b986f7cf5c40', 'f9409754-096c-454e-a9ee-638794597c57', '8337ca37-9b20-4207-8ccd-ec0042d8e5f0']], 'embeddings': None, 'documents': [['“evet” \ndediğini duyacak ve görecekti. Aslında âfâkî ve enfüsî tefekkür ve\naraştırmalardan maksat da budur: Zihin kendi saplantılarından kurtarılacak;\nmefkûreye hürriyet kazandırılacak ve serbest düşünce adesesiyle vicdandaki bu\nince yazılar okunmağa çalışılacak... Bu yolla kalbin derinliklerine bakmağa\nkendini alıştırmış nice kimseler vardır ki, iç müşâhede ve iç duygularıyla, elde\nettikleri vâridâtı, hiçbir kitapta görmeleri mümkün değildir. İlâhî kitapların\nremiz ve işaretleri dahi, ancak bu adese altında kendilerine has renklilikleriyle\nzuhur ederler. Bu ufku göremeyen ve kendini aşamayanlar ise, bundan hiçbir\nzaman, hiçbir şey anlayamayacaklardır.\nŞimdi, gelelim meselenin ikinci yönüne. Acaba bu mukavele ne zaman\nyapıldı? Hemen arz edeyim ki; bu mevzuda, âyet ve

In [42]:
result = []
for e in ergebnisse:
    distances = e['distances'][0]
    
    indices = [idx for idx, value in enumerate(distances) if value < 1]
    if len(indices)>0:
        for idx in indices:
            item = { }
            
            item["ids"] = e['ids'][0][idx]
            item["distances"] = e['distances'][0][idx]
            item["metadatas"] = e['metadatas'][0][idx]
            item["documents"] = e['documents'][0][idx]
            
             
            result.append(item)

            print(item)
            print('****************')


{'ids': '25fdd172-8945-4e0e-bc1f-6bebc6bfdd8b', 'distances': 0.8883215188980103, 'metadatas': {'page': 29, 'source': 'KnowledgeBase/MUZIK_VE_ILAHI_sorularla_islamiyet.pdf'}, 'documents': "Gusül abdesti alırken, gelen\nakıntılar gusle engel olur mu?\nHayır, hiç bir mezhebe göre gusül alırken gelen idrar, kan yellenme gibi\nşeyler gusle zarar vermez; sadece namaz abdestini bozar. \nİlave bilgi için tıklayınız:\nŞafii Mezhebi'ne göre, abdesti bozan durumlar gusle mani mididr?"}
****************
{'ids': 'c1ddcf94-5a4a-4fe7-b63b-3d632241344e', 'distances': 0.9298237562179565, 'metadatas': {'page': 35, 'source': 'KnowledgeBase/MUZIK_VE_ILAHI_sorularla_islamiyet.pdf'}, 'documents': 'yönelik idi.\nNecâset, kan ve dışkı gibi gözle görülen çeşitten ise, bunların temizliği bir\ndefa da olsa pisliğin kendisini gidermekle olur. Ancak, yıkanmasına rağmen\nrenk ve koku gibi giderilmesi güç bir eseri kalırsa, bu zarar vermez. Tercih\nedilen görüşe göre su saf bir hal alıncaya kadar yıkanır. Nitekim Ha

In [43]:
# Distance'a göre sıralama
sorted_list = sorted(result, key=lambda x: x["distances"])

print(sorted_list[:3])


[{'ids': '6d564a15-0b79-482a-a2cf-3ea1fe2b2054', 'distances': 0.663489818572998, 'metadatas': {'page': 250, 'source': 'KnowledgeBase/Diyanet-Vakfi-Yayinlari-ilmihal.pdf'}, 'documents': "abdesti bozar. Vücuttan çıkan kan akmadığı veya çıktığı yerin çevresine\ndağılmadığı sürece abdesti bozmaz. Yaradan çıkan irin ve sarı su da\nböyledir. Çıktığı yerin dışına kendiliğinden dağılmayan bu sıvıların\nsilinmesi halinde de abdest bozulmaz. Şâfiî ve Mâlikîler'e göre idrar ve"}, {'ids': 'fb9301e8-58a8-49c8-a28d-1b01384a53e1', 'distances': 0.7728589773178101, 'metadatas': {'page': 673, 'source': 'KnowledgeBase/Bir-Muslumanin-Yol-Haritasi-Akademi-AHeyeti.pdf'}, 'documents': 'Abdesti Bozan Şeyler\nŞu durumlarda abdest bozulur:\n• İdrar ve dışkı yollarından idrar, meni, mezi, kan gibi bir necasetin,\nherhangi bir sıvının veya maddenin çıkması.\n• Arka taraftan yel çıkması.\n• Ağız ve burundan, ön ile arkadan başka herhangi bir organdan sıvı\nhalinde kan çıkması. Akıcı bir halde ağızdan çıkan kan, tü

In [44]:
sorted_list[:5]

[{'ids': '6d564a15-0b79-482a-a2cf-3ea1fe2b2054',
  'distances': 0.663489818572998,
  'metadatas': {'page': 250,
   'source': 'KnowledgeBase/Diyanet-Vakfi-Yayinlari-ilmihal.pdf'},
  'documents': "abdesti bozar. Vücuttan çıkan kan akmadığı veya çıktığı yerin çevresine\ndağılmadığı sürece abdesti bozmaz. Yaradan çıkan irin ve sarı su da\nböyledir. Çıktığı yerin dışına kendiliğinden dağılmayan bu sıvıların\nsilinmesi halinde de abdest bozulmaz. Şâfiî ve Mâlikîler'e göre idrar ve"},
 {'ids': 'fb9301e8-58a8-49c8-a28d-1b01384a53e1',
  'distances': 0.7728589773178101,
  'metadatas': {'page': 673,
   'source': 'KnowledgeBase/Bir-Muslumanin-Yol-Haritasi-Akademi-AHeyeti.pdf'},
  'documents': 'Abdesti Bozan Şeyler\nŞu durumlarda abdest bozulur:\n• İdrar ve dışkı yollarından idrar, meni, mezi, kan gibi bir necasetin,\nherhangi bir sıvının veya maddenin çıkması.\n• Arka taraftan yel çıkması.\n• Ağız ve burundan, ön ile arkadan başka herhangi bir organdan sıvı\nhalinde kan çıkması. Akıcı bir halde ağ